In [1]:
import getpass
import os

from dotenv import load_dotenv

# Load the .env file
_ = load_dotenv()

In [2]:
from agno.models.huggingface import HuggingFace
from agno.embedder.ollama import OllamaEmbedder
from agno.models.ollama import Ollama
from agno.agent import Agent
from agno.tools.duckduckgo import DuckDuckGoTools



* LLM

In [3]:
model=HuggingFace(
        id="meta-llama/Meta-Llama-3-8B-Instruct",
        api_key=os.getenv("HUGGINGFACE_API_TOKEN"),
)


# model=Ollama(id="llama3.2:3b")


In [4]:
ollama_embedder = OllamaEmbedder(id="nomic-embed-text:latest",dimensions=768)

### Web Searcher Agent

In [12]:
web_searcher = Agent(model=model, 
                     tools=[DuckDuckGoTools()], 
                     name="Web Searcher",  
                     role="Searches the web for information on a topic", 
                     show_tool_calls=True,
                     markdown=False)

response = web_searcher.run("Who is Thomas Edison?")
print(response.content) #latencia 1min 50

Thomas Edison was an American inventor and businessman who developed many important technologies. He is best known for his invention of the light bulb, phonograph, and motion picture camera.


###  RAG Agent

In [6]:
from agno.vectordb.lancedb import LanceDb, SearchType

# Define the database URL where the vector database will be stored
db_url = "./tmp/lancedb"
vector_db = LanceDb(
    table_name="recipes",  # Table name in the vector database
    uri=db_url,  # Location to initiate/create the vector database
    embedder=ollama_embedder,  # Without using this, it will use OpenAIChat embeddings by default
    search_type=SearchType.vector,
)

In [7]:
from agno.knowledge.pdf_url import PDFUrlKnowledgeBase
# Create a knowledge base of PDFs from URLs
knowledge_base = PDFUrlKnowledgeBase(
    urls=["https://agno-public.s3.amazonaws.com/recipes/ThaiRecipes.pdf"],
    vector_db=vector_db
)
# Load the knowledge base
knowledge_base.load(recreate=False)

INFO     Creating collection

INFO     Loading knowledge base

INFO     Reading: https://agno-public.s3.amazonaws.com/recipes/ThaiRecipes.pdf

INFO     Added 0 documents to knowledge base

In [8]:
# Create and use the agent
rag_agent = Agent(model=model,
              name="RAG Agent",
              role="Responds to questions about Thai recipes", 
              knowledge=knowledge_base,  

              # Add a tool to search the knowledge base which enables agentic RAG.
               # This is enabled by default when `knowledge` is provided to the Agent.
              search_knowledge=False,
              show_tool_calls=True,
              markdown=False)

In [9]:

# response = rag_agent.run("How do I make chicken and galangal in coconut milk soup")
# print("response:", response.content)

### Multi-Agent Team

In [10]:
team_leader = Agent(
    name="Multi-Agent Team",
    role="Team leader agent",
    model=model,
    team=[web_searcher,rag_agent],
     instructions=[
        "First, search  for what the user is asking about.",
        "Finally, provide a thoughtful and engaging summary.",
    ],
    show_tool_calls=True,
    markdown=False
)


In [11]:
team_leader.run("Who is Thomas Edison?")

HfHubHTTPError: 422 Client Error: Unprocessable Entity for url: https://api-inference.huggingface.co/models/meta-llama/Meta-Llama-3-8B-Instruct/v1/chat/completions (Request ID: rnr_7Q)

Input validation error: cannot compile regex from schema: 'type' must be a string
Make sure 'text-generation' task is supported by the model.